# Workframe_Data_Preparation:

Merke: die Reihenfolge bei der Data-Preparation ist nicht festgelegt. Je nach Datensatz kann eine andere Reihenfolge sinnvoll sein.


# Ausgangssituation

Du arbeitest als Data Analyst bei der P-2-P-Plattform https://www.kiva.org/, die vor einem Jahr gegründet wurde. Nun wollt ihr euer Geschäft erweitern. Euer Team hat sich aufgeteilt und jeder Analyst hat einen Teilbereich der Daten. Deine Aufgabe ist es in einer **explorativen Datenanalyse** Insights für eure Plattform herauszufinden.

# Crisp-DM

## Geschäftsverständnis:

Euer Geschäftsmodell ist das Betreiben einer Plattform (crowd-investing) bei der sich Personen die eine Geschäftsidee haben, aber nicht das benötigte Geld, anmelden und für ihr Projekt innerhalb einer vorgegebenen Zeit Geld sammeln können. 

Auf der anderen Seite habt ihr Geldgeber, die gern ihr Geld in Projekte anlegen möchten und nach Investitionen suchen. 

Als Vermittler bringt eure Plattform also Geldnehmer und Geldgeber zusammen. 

Deine Datenbasis ist die Historie eurer Plattform.

**Getroffene Annahmen zu dem Geschäftsmodell**

Alle Projekte sind abgeschlossene Projekte, d.h. die Zeit, um für sein Projekt Geld zu sammeln ist abgelaufen. 
Euer Geschäftsmodell sieht es vor, dass die gesammelten Gelder ausgezahlt werden, auch wenn der Zielbetrag nicht erreicht wurde.

## Datenverständnis:

**Data Dictionary**

- funded_amount ... mit Ablauf der "Crowding"zeit erhaltener Betrag/ ausgezahlter Betrag in USD
- loan_amount ... Zielbetrag (Betrag dem man für das Projekt erreichen wollte) in USD
- activity ... Unterkategory zu dem das Ziel des Crowdprojektes thematisch gehört
- sector ... Oberkategory in den das Crowdprojektes Thema fällt
- use ... Kurzbeschreibung wofür das Geld verwendet werden soll
- country_code ... Ländercode nach ISO Norm
- country ... Ländername nach ISO Norm
- region ... Region
- currency ... Währung in der der funded_amount dann ausgezahlt wurde
- term in months ... Dauer über die der Kredit ausgezahlt werden soll
- lender_count ...Darlehensgeber (also wieviele Personen Geld für das Projekt gegeben haben)
- borrower_genders ... Geschlecht und Anzahl der Darlehensnehmer, also diejenigen die das Crowdprojekt initiiert haben *Alle dann ein Geschlecht?*       
- repayment interval ... vertraglich vereinbarte Rückzahlungsmodalitäten/-rhythmus  *Einheit?*

**Welche Informationen für kiva können aus den Daten generiert werden? Was ist das Ziel der Analyse?**

**Brainstorming:**

- Fokus auf Kredite, Geschlecht, Land: 
    * Univariat: Anteil der voll finanzierten Projekte
    * Bivariat: Anteil der voll finanzierten Projekte in Bezug auf Eigenschaften der Kreditnehmer (Geschlecht, mehr als 1 Person)
    * Multivariat: Anteil der voll finanzierten Projekte in Bezug auf das Geschlecht und das Land
- Fokus auf Länder:
    * Univariat: Ranking der einzelnen Länder nach Anzahl der Kredite
    * Bivariat: Ranking der einzelnen activities/ sectors und Länder nach Anzahl der Kredite
    * Multivariat: Länder, activities/ sectors in Abhängigkeit zur gewünschten Kredithöhe/ zum Anteil der voll finanzierten Kredite/ zum Anteil der prozentual erfüllten gewünschten Kredithöhe
- Misserfolgsfaktoren: 
    * Was ist mit funded amount?
    * Wann wird weniger Geld eingenommen als angefordert?
    * Verteilung über Länder, Gender, Beträge Activity, Sector?
- Was sind Erfolgsfaktoren?
    * Wann bekamen Personen mehr Geld als sie erbeten haben?
    * Wann weniger?
- Was ist eigentlich "teuer"?
    * Zusammenhänge zwischen Activity, sector und gesammeltem Geld?
- Wie aktiv sind welche Länder?
    * Gibt es Muster bezüglich sector, activity, repayment_interval, term_in_months?
- Haben größere Kredite mehr Lender, oder tndenziell höhere individuelle Beiträge?
- Wie aktiv sind welche Währungen?


**Entscheidung für ein Analysethema:**

*Entscheidung für das Analysethema erfolgt eigentlich nach den ersten Datenmanipulationen. Der Grund dafür ist, dass zu einem solch frühen Zeitpunkt noch gar keine fundierte Entscheidung getroffen werden kann, was genau analysiert werden soll. Die Entscheidung ist hier lokalisiert, da es thematisch am besten passt.* 

*Ziel:*

Anteil der voll finanzierten Projekte, Gründe für Nicht-/ oder nur Teilfinanzierung:

*Zielerreichung*

- Univariate Analyse: Anteil der voll-, teil- und unfinanzierten Projekte
- Dann Kreuztabelle --> welche Anhaltspunkte gibt es in numerischen Tabellen?
- Bivariate Analyse: Anteil der voll-, teil- und unfinanzierten Projekte in Abhängigkeit von einem weiteren signifikanten Parameter, alternativ Zusammenhang mit Geschlecht? Zusammenhang mit bestimmten Ländern? Zusammenhang mit Activity/ Sector? Zusammenhang mit repayment_interval?
- Multivariat, wenn bivariat abgeschlossen. 



## Datenaufbereitung:

Zunächst Import der für die Analyse relevanten Pakete:

In [120]:
import numpy as np
import pandas as pd
import matplotlib as mp
import plotly.express as px
import re

### Datensatz einlesen

**Je nach Datensatztyp**

#### .csv:

In [121]:
# zunächst "Probeladen" => Feststellung des Trennzeichens, was ist die Indexspalte?

df_p2p = pd.read_csv(
                      'data_abschlussprojekt.csv',             # Dateiname reicht aus, wenn Datei im gleichen Ordner ist wie Skript
                      delimiter='/n',                 # Standard-Zeilentrennzeichen, so dass nicht alles in einer Zeile angezeigt wird
                      engine='python',                 
                      nrows=2,                        # nur 2 Zeilen, zum "reingucken"
                      )
df_p2p

# => Trennzeichen '#'



,# funded_amount# loan_amount# activity# sector# use# country_code# country# region# currency# term_in_months# lender_count# borrower_genders# repayment_interval
0,0#300.0#300.0#Fruits & Vegetables#Food#To buy ...
1,1#575.0#575.0#Rickshaw#Transportation#to repai...


In [122]:
# "eigentliches" Laden der Daten mit dem richtigen Trennzeichen und der richtigen Indexspalte

df_p2p = pd.read_csv('data_abschlussprojekt.csv',             # Dateiname reicht aus, wenn Datei im gleichen Ordner ist wie Skript
                      sep='#',
                      index_col=0
                      )
df_p2p 

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular
2,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,43.0,6,female,bullet
3,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,11.0,8,female,irregular
4,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,14.0,16,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671200,0.0,25.0,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",PY,Paraguay,Concepción,USD,13.0,0,female,monthly
671201,25.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,1,female,monthly
671202,0.0,25.0,Games,Entertainment,NaN,KE,Kenya,NaN,KES,13.0,0,NaN,monthly
671203,0.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,0,female,monthly


### Objekt kennenlernen

In [123]:
# Spalten anzeigen

df_p2p.columns

Index([' funded_amount', ' loan_amount', ' activity', ' sector', ' use',
       ' country_code', ' country', ' region', ' currency', ' term_in_months',
       ' lender_count', ' borrower_genders', ' repayment_interval'],
      dtype='object')

*Ergebnis*

Sind Leerzeichen am Anfang, oder am Ende der Spaltennamen? => Werden entfernt

In [124]:
# Leerzeichen entfernen in den Spaltennamen

df_p2p.columns = df_p2p.columns.str.strip() 

# Wurden die Leerzeichen wirklich entfernt?

df_p2p.columns

Index(['funded_amount', 'loan_amount', 'activity', 'sector', 'use',
       'country_code', 'country', 'region', 'currency', 'term_in_months',
       'lender_count', 'borrower_genders', 'repayment_interval'],
      dtype='object')

Sind Leerzeichen in einzelnen Zellen von 'Object'-Spalten? => Werden entfernt

In [125]:
# potenzielle Leerzeichen von strings innerhalb der Spalten entfernen

def trim_object_columns(df):
    
    # Identifizierung aller Spalten vom Datentyp 'object'
    object_columns = df.select_dtypes(include=['object']).columns
    
    # Entfernung potenziellet Leerzeichen am Anfang und Ende der Werte in diesen Spalten
    df.loc[:,object_columns] = df.loc[:,object_columns].apply(lambda col: col.strip() if isinstance(col, str) else col)
    
    return df

In [126]:
# Anwendung der Funktion:

df_p2p = trim_object_columns(df_p2p)
df_p2p

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular
2,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,43.0,6,female,bullet
3,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,11.0,8,female,irregular
4,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,14.0,16,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671200,0.0,25.0,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",PY,Paraguay,Concepción,USD,13.0,0,female,monthly
671201,25.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,1,female,monthly
671202,0.0,25.0,Games,Entertainment,NaN,KE,Kenya,NaN,KES,13.0,0,NaN,monthly
671203,0.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,0,female,monthly


#### Enthaltene Datentypen

In [127]:
# Datentypen

df_p2p.dtypes

funded_amount         float64
loan_amount           float64
activity               object
sector                 object
use                    object
country_code           object
country                object
region                 object
currency               object
term_in_months        float64
lender_count            int64
borrower_genders       object
repayment_interval     object
dtype: object

**Auswertung der Datentypen**

    Spalte              |   techn. Datentyp       |   fachl. Datentyp     |    Beurteilung
    --------------------|-------------------------|-----------------------|------------------------------------------------
    funded_amount       |     float 64            |       float64         |   ok
    loan_amount         |     float 64            |       float64         |   ok
    activity            |     object              |       object          |   Werteausprägungen --> category
    sector              |     object              |       object          |   Werteausprägungen --> category
    use                 |     object              |       object          |   ok
    country_code        |     object              |       object          |   8 Missing Values - kongruent zur folgenden Spalte: Löschen?
    country             |     object              |       object          |   enthält 87 Werte --> category
    region              |     object              |       object          |   würde sich auch für category anbieten
    currency            |     object              |       object          |   würde sich für category anbieten (167 verschiedene Einträge)
    term_in_months      |     float64             |       int64           |   es gibt nur ganze Monate -> Spalte ansehen!
    lender_count        |     int64               |       int64           |   ok
    borrower_genders    |     object              |       object          |   ok
    repaymeent interval |     object              |       object          |   enthält 4 Werte, hier bietet sich category an

#### Erste kleine Statistik:

Wertebereiche der numerischen Spalten

In [128]:
# describe() nimmt nur die spalten mit numerischen Werten
 
df_p2p.describe()

,funded_amount,loan_amount,term_in_months,lender_count
count,671205.000000,671205.000000,671205.000000,671205.000000
mean,785.995061,842.397107,13.739022,20.590922
std,1130.398941,1198.660073,8.598919,28.459551
min,0.000000,25.000000,1.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000
50%,450.000000,500.000000,13.000000,13.000000
75%,900.000000,1000.000000,14.000000,24.000000
max,100000.000000,100000.000000,158.000000,2986.000000


**Auswertung der kleinen Statistik:**

        Spalte    |      Wertebereich    |    Kommentar
    ----------------------------------------------------------------------
    funded_amount |   0.00-100,000.00    |      sieht ok aus  
    loan_amount   |   25 - 100,000.00    |      sieht ok aus
    term_in_months|     1.0 - 158.0      |      sieht ok aus
    lender_count  |         0-2986       |      sieht ok aus

Werteanzahl aller Spalten:



In [129]:
df_p2p.nunique()

funded_amount            610
loan_amount              479
activity                 163
sector                    15
use                   424912
country_code              86
country                   87
region                 12695
currency                  67
term_in_months           148
lender_count             503
borrower_genders       11298
repayment_interval         4
dtype: int64

### Duplikate

#### Definition:

Ein Duplikat ist für mich eine Kombination gleicher Werte aus allen Spalten.

#### Ermittlung gemäß Definition:

In [130]:
# Alle Spalten werden 

duplicates = df_p2p.loc[df_p2p.duplicated(keep=False), :]
duplicates

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
327,275.0,275.0,Farming,Agriculture,to buy fertilizers and other farm supplies.,PH,Philippines,"Brookes Point, Palawan",PHP,8.0,8,female,irregular
392,100.0,100.0,Home Energy,Personal Use,to buy a solar lamp.,SV,El Salvador,NaN,USD,14.0,4,male,monthly
405,100.0,100.0,Home Energy,Personal Use,to buy a solar-powered lamp.,SV,El Salvador,NaN,USD,14.0,4,male,monthly
498,100.0,100.0,Home Energy,Personal Use,to buy a solar-powered lamp.,SV,El Salvador,NaN,USD,14.0,4,male,monthly
606,100.0,100.0,Home Energy,Personal Use,to buy a solar-powered lamp.,SV,El Salvador,NaN,USD,14.0,4,male,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671200,0.0,25.0,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",PY,Paraguay,Concepción,USD,13.0,0,female,monthly
671201,25.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,1,female,monthly
671202,0.0,25.0,Games,Entertainment,NaN,KE,Kenya,NaN,KES,13.0,0,NaN,monthly
671203,0.0,25.0,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",KE,Kenya,NaN,KES,13.0,0,female,monthly


#### Umgang gemäß Definition

Die Objekte in der Liste haben keine erkennbaren Unterscheidungsmerkmale. Mit 34633 Zeilen wäre die Anzahl der Duplikate mit ca. 5% hoch.

Auch wenn die Zeilen komplett gleich sind, kann ich nicht sagen, ob es sich hierbei um einen oder mehrere Vorgänge handelt. Zur tatsächlichen Unterscheidung bezüglich Duplikate fehlt etwas wie eine Kreditnehmer-ID, oder auch ein Zeitstempel, wann das Kreditersuchen gestartet wrude. 

Da ich mir nicht sicher sein kann und die (relative) Anzahl ziemlich hoch wäre, lösche ich keine Einträge.

**Annahme für nachfolgende Arbeitsschritte:**
Ich nehme an, dass in ähnlichen Ländern ähnliche Probleme existieren, so dass es sehr wahrscheinlich ist, dass Individuen bei der Finanzierung ihrer Vorhaben vor ähnlichen Problemen stehen. 

### Fehlende Werte:

#### Identifizierung

Kurze Überprüfung systemseitig fehlende Werte:



In [131]:
df_p2p.isnull().sum()

funded_amount             0
loan_amount               0
activity                  0
sector                    0
use                    4232
country_code              8
country                   0
region                56800
currency                  0
term_in_months            0
lender_count              0
borrower_genders       4221
repayment_interval        0
dtype: int64

##### Synonyme

Die vorangegangene Untersuchung der Spalten mit .dtypes hat ergeben, dass bei den numerischen Spalten keine Synonyme verwendet werden. Eine zusätzliche Überprüfung ist somit nicht notwendig.

Bei den Spalten, die den Datentyp 'object' enthalten überprüfe ich mit einer Funktion:

##### Ermittlung

In [132]:
def is_potential_none(lower_string):
        patterns_none = [
            r'^-$', r'^\?$', r'^none$', r'^null$', r'^n.a.$', r'^n/a$',
            r'^(miss|no\s{1}|not\s{1})', r'^empty$', r'^void$', r'^undefined$',
            r'^nil$', r'^blank$', r'^0*$', r'^0{1}\.{1}0+$'
        ]
        return any(re.search(p, lower_string) for p in patterns_none)

def set_potential_none():
        potential_none = {
            col_name: [value for value in col_data.unique() if is_potential_none(str(value).lower().strip())]
            for col_name, col_data in df_p2p.items()
        }
        return potential_none

set_potential_none()

{'funded_amount': [0.0],
 'loan_amount': [],
 'activity': [],
 'sector': [],
 'use': [' ', 'none'],
 'country_code': [],
 'country': [],
 'region': ['Missirikoro', 'Missasso', 'Missala'],
 'currency': [],
 'term_in_months': [],
 'lender_count': [0],
 'borrower_genders': [],
 'repayment_interval': []}

**Auswertung**

- funded_amount: Ersetzen nicht notwendig, da '0.0' vorkommen darf
- use: ' ' und 'none' ersetzen durch NaN
- region: Ausdrücke sind keine Synonyme für NaN-Werte
- lender_count: 0 - Werte dürfen ebenfalls vorkommen

##### Synonym ersetzen

Ersetzen der Synonyme mit 'NaN'

In [133]:
df_p2p.replace(to_replace={'use': ' ', 'use': 'none'}, value=np.nan, inplace=True)

df_p2p.isna().sum()

funded_amount             0
loan_amount               0
activity                  0
sector                    0
use                    4233
country_code              8
country                   0
region                56800
currency                  0
term_in_months            0
lender_count              0
borrower_genders       4221
repayment_interval        0
dtype: int64

##### Systemseitig erkannte fehlende Werte

Bei der Überprüfung der Ersetzung fehlender Werte wurden die systemseitig fehlenden Werte schon abgefragt. Nun stelle ich noch den prozentualen Anteil der fehlenden Werte fest:

In [134]:
round(df_p2p.isnull().mean()*100,4).sort_values(ascending=False) 

region                8.4624
use                   0.6307
borrower_genders      0.6289
country_code          0.0012
funded_amount         0.0000
loan_amount           0.0000
activity              0.0000
sector                0.0000
country               0.0000
currency              0.0000
term_in_months        0.0000
lender_count          0.0000
repayment_interval    0.0000
dtype: float64

#### Umgang

**Erste Spalte mit fehlenden Werten ('use')**

Spalte 'use' eignet sich durch die Variabilität (> 400,000 unique values) nicht für eine Analyse (Müsste durch Text Mining geclustered werden). Ersatzweise dienen die Spalten 'activity' und 'sector' wenn nötig zur groben Kategorisierung.

Deshalb kann die Spalte 'use' gelöscht werden.

In [135]:
# z.B. nach Löschen einer Spalte:

df_p2p.drop(columns=['use'], inplace=True)

# check ob Löschung funktioniert hat:

'use' in df_p2p

# --> Ergebnis muss 'False' sein

False

**Zweite Spalte mit fehlenden Werten ('region')** 

Spalte 'region' weist mit >8% einen sehr hohen Anteil fehlender Werte auf. Die Spalte wird auch für keine geplante Analyse mehr benötigt. Diese sollen sich lediglich auf die Länder beziehen.


In [136]:
# z.B. nach Löschen einer Spalte:

df_p2p.drop(columns=['region'], inplace=True)

# check ob Löschung funktioniert hat:

'region' in df_p2p

# --> Ergebnis muss 'False' sein

False



**Dritte Spalte mit fehlenden Werten ('borrower_genders')**

Spalte 'borrower_genders' ist für weitere Analysen wichtig. Das Löschen dieser Spalte kommt somit nicht in Frage. Der Anteil der NaN-Werte liegt < 1%. Diese könnten somit bedenkenlos gelöscht werden. Ersetzen kommt hier nicht in Frage, da wir keine Informationen zu den Kreditnehmern haben. 
**Wichtig:** Die behaltenen Zeilen müssen weiter transformiert werden. Die enthaltenen Strings sind für Analysen ungeeignet!

*Zeilen mit NaN-Werten löschen*

In [137]:
df_p2p = df_p2p.loc[df_p2p.loc[:, 'borrower_genders'].notna()]
df_p2p

,funded_amount,loan_amount,activity,sector,country_code,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,300.0,300.0,Fruits & Vegetables,Food,PK,Pakistan,PKR,12.0,12,female,irregular
1,575.0,575.0,Rickshaw,Transportation,PK,Pakistan,PKR,11.0,14,"female, female",irregular
2,150.0,150.0,Transportation,Transportation,IN,India,INR,43.0,6,female,bullet
3,200.0,200.0,Embroidery,Arts,PK,Pakistan,PKR,11.0,8,female,irregular
4,400.0,400.0,Milk Sales,Food,PK,Pakistan,PKR,14.0,16,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...
671199,0.0,25.0,Livestock,Agriculture,PY,Paraguay,USD,13.0,0,female,monthly
671200,0.0,25.0,Livestock,Agriculture,PY,Paraguay,USD,13.0,0,female,monthly
671201,25.0,25.0,Livestock,Agriculture,KE,Kenya,KES,13.0,1,female,monthly
671203,0.0,25.0,Livestock,Agriculture,KE,Kenya,KES,13.0,0,female,monthly


**Vierte Spalte mit fehlenden Werten ('country_code')**

Spalte 'country_code' hat eine direkte Abhängigkeit zur Spalte 'country'. Daher kann 'country_code' gelöscht werden. Für Analysen ist der eigentliche Ländername eindrücklicher als dessen Kürzel. 

In [138]:
# z.B. nach Löschen einer Spalte:

df_p2p.drop(columns=['country_code'], inplace=True)

# check ob Löschung funktioniert hat:

'country_code' in df_p2p

# --> Ergebnis muss 'False' sein

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\1396725459.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



False

### Ausreißer

Ziel der Analyse ist es, Gründe für Finanzierung/ Nicht-Finanzierung herauszufinden. 

Deshalb konzentriere ich mich bei den Ausreißern auch auf die Spalten 'loan_amount' und 'funded_amount'.

Um ein vollumfängliches Bild zu bekommen, könnten auch Ausreißer nützlich sein. Die 'erste kleine Statistik' hat gezeigt, dass Ausreißer wohl einen hohen Einfluss auf die übrigen Werte haben. Die 'erste kleine Statistik' lässt bei 'funded_amount', sowie 'loan_amount' eine Gamma-Verteilung vermuten, dies gilt es visuell zu überprüfen.

Zunächst ermittle ich die Ausreißer. Ich bilde mit diesen eine eigene Gruppe (hier: Variable), die ich ggf. (wenn ausreichend Zeit bleibt) gesondert analysiere. Da mich eher das 'Big Picture' - also die Mehrheit der Kredite interessiert, schließe ich die Ausreißer von der eigentlichen Analyse aus.

#### Visuelle Überprüfung der Verteilung

Spalten 'loan_amount', 'funded_amount'

In [139]:
# Häufigkeiten für 'loan_amount' und 'funded_amount' berechnen
loan_count = df_p2p['loan_amount'].value_counts().sort_index()
funded_count = df_p2p['funded_amount'].value_counts().sort_index()

# Beide Häufigkeiten in einem DataFrame zusammenführen
df_loan_funded = pd.DataFrame({
    'value': loan_count.index.union(funded_count.index),  # Alle einzigartigen Geldmengen
    'loan_amount_count': loan_count,
    'funded_amount_count': funded_count
}).fillna(0).astype(int)
df_loan_funded

,value,loan_amount_count,funded_amount_count
0.0,0,0,3254
5.0,5,0,7
10.0,10,0,12
15.0,15,0,1
20.0,20,0,8
...,...,...,...
49750.0,49750,1,0
49925.0,49925,1,1
49950.0,49950,1,1
50000.0,50000,36,31


Umwandlung in einen visuell darstellbaren Datensatz

In [140]:
df_long = df_loan_funded.melt(id_vars='value', value_vars=['loan_amount_count', 'funded_amount_count'],
                  var_name='Type', value_name='Count')
df_long

,value,Type,Count
0,0,loan_amount_count,0
1,5,loan_amount_count,0
2,10,loan_amount_count,0
3,15,loan_amount_count,0
4,20,loan_amount_count,0
...,...,...,...
1201,49750,funded_amount_count,0
1202,49925,funded_amount_count,1
1203,49950,funded_amount_count,1
1204,50000,funded_amount_count,31


Visualisierung der Verteilung

In [141]:
# Liniendiagramm erstellen
fig_funded = px.line(
              df_long, x='value', y='Count', 
              labels={'value': 'Geldmenge (US $)', 'Count': 'Häufigkeit'},
              title='Verteilung der Geldmenge',
              color='Type',
              )

# Diagramm anzeigen
fig_funded.show()

Die Hypothese, dass es sich um eine Gamma-Verteilung handeln könnte wurde nun visuell bestätigt. 

Die Ausreißer sind für sich genommen interessant, verzerren aber das Gesamtbild. 

Nun gilt es im nächsten Schritt die Ausreißer im Einzelnen zu identifizieren und im Analysedatensatz zu eliminieren.


#### Ausreißer identifizieren

Mit der Funktion definiere ich Ausreißer wie im Boxplot:

In [142]:
def detect_outliers_iqr(df):
    
    # Schleife über alle Spalten im DataFrame
    
    for column in df.select_dtypes(exclude=["object", "bool", "category"]):
        # Berechnung der Quartile und des IQRs
        Q1 = df.loc[:,column].quantile(0.25)
        Q3 = df.loc[:,column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Definition von Schwellenwerten für Ausreißer
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Erstellung der Ausreißer-Spalte für diese numerische Spalte
        outlier_column_name = f"{column}_outlier"
        df.loc[:,outlier_column_name] = ((df.loc[:,column] < lower_bound) | (df.loc[:,column] > upper_bound)).astype(int)
    
    return df

Problem: der ursprüngliche df wird hier auch verändert. Wie bekomme ich es hin, dass nur der df_outlier als veränderter df auftauchen?

**Funktion anwenden**

In [159]:
df_p2p_outlier = detect_outliers_iqr(df_p2p.copy())
df_p2p_outlier

,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,...,number_males,borrower_count,borrower_gender_cat,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier,number_females_outlier,number_males_outlier,borrower_count_outlier
0,300.0,300.0,Fruits & Vegetables,Food,Pakistan,PKR,12,12,female,irregular,...,0,1,female,0,0,0,0,0,0,0
1,575.0,575.0,Rickshaw,Transportation,Pakistan,PKR,11,14,"female, female",irregular,...,0,2,female,0,0,0,0,1,0,1
2,150.0,150.0,Transportation,Transportation,India,INR,43,6,female,bullet,...,0,1,female,0,0,1,0,0,0,0
3,200.0,200.0,Embroidery,Arts,Pakistan,PKR,11,8,female,irregular,...,0,1,female,0,0,0,0,0,0,0
4,400.0,400.0,Milk Sales,Food,Pakistan,PKR,14,16,female,monthly,...,0,1,female,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,female,monthly,...,0,1,female,0,0,0,0,0,0,0
671200,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,female,monthly,...,0,1,female,0,0,0,0,0,0,0
671201,25.0,25.0,Livestock,Agriculture,Kenya,KES,13,1,female,monthly,...,0,1,female,0,0,0,0,0,0,0
671203,0.0,25.0,Livestock,Agriculture,Kenya,KES,13,0,female,monthly,...,0,1,female,0,0,0,0,0,0,0


**Zeilen mit Ausreißern anzeigen**

In [144]:
# Zeilen anzeigen, die Ausreißer enthalten

df_p2p_outlier.loc[(df_p2p_outlier.loc[:,"funded_amount_outlier"] > 0) | (df_p2p_outlier.loc[:,'loan_amount_outlier'] > 0),:]

,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
35,2225.0,2225.0,Personal Products Sales,Retail,Pakistan,PKR,11.0,58,"female, female, female, female, female, female...",irregular,1,1,0,1
50,3175.0,3175.0,Butcher Shop,Food,Tanzania,TZS,10.0,93,"male, male, male, male, male",monthly,1,1,0,1
59,3175.0,3175.0,Home Products Sales,Retail,Tanzania,TZS,8.0,114,"male, female, female, female",irregular,1,1,0,1
62,4275.0,4275.0,Personal Housing Expenses,Housing,Vietnam,VND,14.0,144,"female, female, female, female, female, female...",bullet,1,1,0,1
70,2000.0,2000.0,Retail,Retail,Iraq,USD,15.0,71,male,monthly,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671115,2650.0,3100.0,Food,Food,Paraguay,PYG,8.0,29,"female, female, female, female, female, female...",monthly,1,1,0,0
671124,1825.0,2975.0,Retail,Retail,Paraguay,PYG,8.0,20,"female, female, female, female, female, female...",monthly,0,1,0,0
671126,1850.0,3725.0,Clothing,Clothing,Paraguay,PYG,8.0,19,"female, female, female, female, female, female...",monthly,0,1,0,0
671132,0.0,3425.0,Personal Medical Expenses,Health,Tajikistan,TJS,20.0,0,female,monthly,0,1,0,0


##### Auswertung (1)

Als Ausreißer gelten 59.194 Einträge. Das sind ca. 10% der Einträge und somit sehr viele. Alle diese Ausreißer auszuschließen würde die Analyse wiederum nach unten verzerren.

Ich überprüfe die Hypothese der Verzerrung nun rechnerisch:

In [145]:
df_p2p_outlier.loc[df_p2p_outlier.loc[:,'loan_amount_outlier'] <= 0,:].describe()

,funded_amount,loan_amount,term_in_months,lender_count,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
count,616556.000000,616556.000000,616556.000000,616556.000000,616556.000000,616556.0,616556.000000,616556.000000
mean,552.607468,584.640122,13.496952,15.987751,0.014218,0.0,0.084148,0.032831
std,422.570894,437.268303,7.836174,13.360508,0.118387,0.0,0.277610,0.178194
min,0.000000,25.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,250.000000,250.000000,8.000000,7.000000,0.000000,0.0,0.000000,0.000000
50%,425.000000,450.000000,13.000000,12.000000,0.000000,0.0,0.000000,0.000000
75%,750.000000,800.000000,14.000000,21.000000,0.000000,0.0,0.000000,0.000000
max,2075.000000,2075.000000,141.000000,137.000000,1.000000,0.0,1.000000,1.000000


Es gibt keine Standards, wie sich selbst robuste statistische Maßzahlen verändern dürfen. Allerdings kann man sagen, dass eine Veränderung erheblich ist, wenn sich der IQR um mehr als 10-15% verändert.

Der IQR ändert sich bei Ausschluss aller Ausreißer gemäß Boxplot-Definition um 31%. Somit wird der Charakter des Datensatzes beeinflusst.

Wie kann ich sonst auf ausschließbare Werte kommen?

- Bei der visuellen Gamma-Verteilung im Plot kann ich ablesen, dass es eine letzte signifikante Häufung der Werte im Bereich 10.000 USD gibt. Ich sehe mir nun an, wie viele Werte ich ausschließen würde wenn ich die Grenze bei >10.000 USD setze

In [146]:
df_p2p_outlier.loc[df_p2p_outlier.loc[:, 'loan_amount'] > 10000, :]

,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
8536,12925.0,12925.0,Renewable Energy Products,Retail,India,INR,14.0,408,male,irregular,1,1,0,1
31000,10150.0,10150.0,Home Energy,Personal Use,Kenya,KES,86.0,344,"male, male, female, male",irregular,1,1,1,1
32108,11650.0,11650.0,Bakery,Food,Yemen,YER,15.0,312,male,monthly,1,1,0,1
32424,11775.0,11775.0,Metal Shop,Manufacturing,Rwanda,USD,14.0,398,male,monthly,1,1,0,1
34196,50000.0,50000.0,Renewable Energy Products,Retail,Peru,USD,14.0,1446,male,irregular,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659727,13225.0,50000.0,Arts,Arts,Uganda,USD,14.0,447,male,irregular,1,1,0,1
660211,25000.0,25000.0,Food,Food,Ghana,USD,14.0,785,female,irregular,1,1,0,1
660807,46125.0,50000.0,Renewable Energy Products,Retail,Pakistan,USD,14.0,805,female,irregular,1,1,0,1
662660,30000.0,30000.0,Renewable Energy Products,Retail,Haiti,USD,26.0,969,female,irregular,1,1,1,1


Der Einfluss, den 173 Zeilen auf die Grundgesamtheit haben können ist schwer abzuschätzen. Deshalb überprüfe ich rechnerisch wie sich der Datensatz ohne diese Werte verhalten würde. 

In [147]:
df_p2p_outlier.loc[df_p2p_outlier.loc[:, 'loan_amount'] <= 10000, :].describe()

,funded_amount,loan_amount,term_in_months,lender_count,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
count,666811.000000,666811.000000,666811.000000,666811.000000,666811.000000,666811.000000,666811.000000,666811.000000
mean,778.560657,833.393120,13.724592,20.360799,0.081611,0.075366,0.097672,0.080062
std,1020.285205,1077.764117,8.569426,24.669958,0.273771,0.263981,0.296871,0.271389
min,0.000000,25.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,450.000000,500.000000,13.000000,13.000000,0.000000,0.000000,0.000000,0.000000
75%,900.000000,1000.000000,14.000000,24.000000,0.000000,0.000000,0.000000,0.000000
max,10000.000000,10000.000000,158.000000,507.000000,1.000000,1.000000,1.000000,1.000000


##### Auswertung (2)

Im Vergleich zur "ersten kleinen Statistik" haben sich weder Mittelwert, noch Standardabweichunggroßartig geändert. Der IQR ist bei dieser Grenze genau gleich geblieben.

Allerdings möchte ich die 10-15% Veränderung gerne ausreizen, um mich auf eine signifikante Grundgesamtheit beziehen zu können. Deshalb setze ich anschließend die Grenze herab:

Ich möchte überprüfen, welchen Einfluss es hat, wenn ich die Grenze auf 5.000 USD verschiebe:

In [148]:
df_p2p_outlier.loc[df_p2p_outlier.loc[:, 'loan_amount'] > 5000, :]

,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
88,5475.0,5475.0,Retail,Retail,Tanzania,TZS,9.0,169,"female, female, female, female, female, female...",irregular,1,1,0,1
93,5775.0,5775.0,Charcoal Sales,Retail,Tanzania,TZS,6.0,154,"male, male, female, female, male, male, female...",irregular,1,1,0,1
231,10000.0,10000.0,Food Production/Sales,Food,United States,USD,36.0,280,female,monthly,1,1,1,1
246,5100.0,5100.0,Food Stall,Food,Tanzania,TZS,8.0,171,"female, female, female, female, female, female...",irregular,1,1,0,1
371,5925.0,5925.0,Services,Services,Tanzania,TZS,9.0,134,"female, female, female, female, male, female, ...",irregular,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670997,6025.0,6025.0,Food,Food,Paraguay,PYG,5.0,11,"female, female, female, female, female, female...",irregular,1,1,0,0
671000,175.0,6250.0,Clothing Sales,Clothing,Bolivia,BOB,11.0,4,"female, female, female, female, female, female...",monthly,0,1,0,0
671010,1525.0,6450.0,Crafts,Arts,Paraguay,PYG,8.0,21,"female, female, female, female, female, female...",monthly,0,1,0,0
671011,4925.0,6100.0,Food Market,Food,Bolivia,BOB,9.0,26,"female, female, female, female, female, female...",monthly,1,1,0,0


Würde ich die Grenze bei 5.000 USD setzen, würde ich annähernd 10.000 Zeilen ausschließen. Wie wirkt sich das statistisch aus?

In [149]:
df_p2p_outlier.loc[df_p2p_outlier.loc[:, 'loan_amount'] <= 5000, :].describe()

,funded_amount,loan_amount,term_in_months,lender_count,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
count,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000
mean,703.013864,749.622628,13.685618,19.137014,0.069415,0.062284,0.095501,0.071363
std,762.109132,797.678505,8.351952,20.534151,0.254159,0.241670,0.293907,0.257431
min,0.000000,25.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,450.000000,475.000000,13.000000,13.000000,0.000000,0.000000,0.000000,0.000000
75%,875.000000,950.000000,14.000000,24.000000,0.000000,0.000000,0.000000,0.000000
max,5000.000000,5000.000000,141.000000,345.000000,1.000000,1.000000,1.000000,1.000000


##### Auswertung (3)

Mittelwert und Standardabweichung haben sich erwartungsgemäß angenähert. der IQR beträgt jetzt 675 zu ursprünglich 725. Das ist eine Änderung um 9,3% und somit innerhalb des akzeptablen Rahmens.

Nun schließe ich alle Werte aus, die in der Spalte loan_amount > 5.000 USD betragen:

In [150]:
df_p2p = df_p2p.loc[df_p2p.loc[:, 'loan_amount']<= 5000]
df_p2p.describe()

,funded_amount,loan_amount,term_in_months,lender_count,funded_amount_outlier,loan_amount_outlier,term_in_months_outlier,lender_count_outlier
count,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000,657508.000000
mean,703.013864,749.622628,13.685618,19.137014,0.069415,0.062284,0.095501,0.071363
std,762.109132,797.678505,8.351952,20.534151,0.254159,0.241670,0.293907,0.257431
min,0.000000,25.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,450.000000,475.000000,13.000000,13.000000,0.000000,0.000000,0.000000,0.000000
75%,875.000000,950.000000,14.000000,24.000000,0.000000,0.000000,0.000000,0.000000
max,5000.000000,5000.000000,141.000000,345.000000,1.000000,1.000000,1.000000,1.000000


### Daten gruppieren/ aggregieren





### Daten kategorisieren

#### Einfach zu kategorisierende Daten:

Zur Kategorisierung eignen sich mehrere Spalten:

- activity
- sector
- country 
- borrower_genders
- repayment_interval

Bis auf die Spalte 'borrower_genders' kann dies auch einfach über die Umwandlung des Datentyps realisiert werden. Die Kategorien an sich sind schon vorhanden:


In [153]:
# Umzuwandelnde Spalten definieren

columns_to_convert = ['activity', 'sector', 'country', 'currency','repayment_interval']

# Spalten umwandeln:

df_p2p.loc[:, columns_to_convert] = df_p2p.loc[:, columns_to_convert].astype('category')
df_p2p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 657508 entries, 0 to 671204
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   funded_amount       657508 non-null  float64 
 1   loan_amount         657508 non-null  float64 
 2   activity            657508 non-null  category
 3   sector              657508 non-null  category
 4   country             657508 non-null  category
 5   currency            657508 non-null  category
 6   term_in_months      657508 non-null  float64 
 7   lender_count        657508 non-null  int64   
 8   borrower_genders    657508 non-null  object  
 9   repayment_interval  657508 non-null  category
dtypes: category(5), float64(3), int64(1), object(1)
memory usage: 33.9+ MB


C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\3513836828.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\3513836828.py:7: DeprecationWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



Die Spalte term_in_months ist immer noch float64, obwohl es int sein müsste. Das korrigiere ich nun:

In [154]:
df_p2p.loc[:, 'term_in_months'] = df_p2p.loc[:, 'term_in_months'].astype('int')
df_p2p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 657508 entries, 0 to 671204
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   funded_amount       657508 non-null  float64 
 1   loan_amount         657508 non-null  float64 
 2   activity            657508 non-null  category
 3   sector              657508 non-null  category
 4   country             657508 non-null  category
 5   currency            657508 non-null  category
 6   term_in_months      657508 non-null  int32   
 7   lender_count        657508 non-null  int64   
 8   borrower_genders    657508 non-null  object  
 9   repayment_interval  657508 non-null  category
dtypes: category(5), float64(2), int32(1), int64(1), object(1)
memory usage: 31.4+ MB


C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\2569928276.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\2569928276.py:1: DeprecationWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



#### Daten, deren Kategorisierung komplexer ist:

Die Spalte borrower_genders ist nicht so einfach zu kategorisieren. Sie erfüllt eigentlich mehrere Zwecke:

- Information über das Geschlecht der Kreditnehmer
- Anzahl der Kreditnehmer

Hier sind zwei Spalten zu ergänzen:
- Kategorie: 'female', 'male', 'mixed' --> category
- Anzahl der Kreditnehmer --> int 

Folglich muss sowohl die Anzahl der Nennungen, als auch welche Geschlechtsnennungen erfolgt sind aus der Spalte borrower_genders extrahiert werden.

Im Anschluss kann die Spalte borrower_genders gelöscht werden.

Zunächst gehe ich einen Umweg über die Zählung der Geschlechter (einzeln) je Zelle in borrower_genders:

In [155]:
# Zunächst ergänze ich eine Spalte number_females:

df_p2p['number_females'] = df_p2p['borrower_genders'].apply(lambda x: x.split(', ').count('female'))

# und eine Spalte number_males:

df_p2p['number_males'] = df_p2p['borrower_genders'].apply(lambda x: x.split(', ').count('male'))

# sowie eine Spalte lender_count:

df_p2p['borrower_count'] = df_p2p['number_females'] + df_p2p['number_males']

df_p2p 

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\1598573086.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\1598573086.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\1598573086.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,number_females,number_males,borrower_count
0,300.0,300.0,Fruits & Vegetables,Food,Pakistan,PKR,12,12,female,irregular,1,0,1
1,575.0,575.0,Rickshaw,Transportation,Pakistan,PKR,11,14,"female, female",irregular,2,0,2
2,150.0,150.0,Transportation,Transportation,India,INR,43,6,female,bullet,1,0,1
3,200.0,200.0,Embroidery,Arts,Pakistan,PKR,11,8,female,irregular,1,0,1
4,400.0,400.0,Milk Sales,Food,Pakistan,PKR,14,16,female,monthly,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,female,monthly,1,0,1
671200,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,female,monthly,1,0,1
671201,25.0,25.0,Livestock,Agriculture,Kenya,KES,13,1,female,monthly,1,0,1
671203,0.0,25.0,Livestock,Agriculture,Kenya,KES,13,0,female,monthly,1,0,1


Nachdem ich weiß wie viele Individuen eines jeden Geschlechtes beteiligt sind, fasse ich number_female und number_male zu einer Spalte als Kategorie zusammen:

In [158]:
# Neue Spalte 'borrower_gender_cat' basierend auf 'number_females' und 'number_males' erstellen
df_p2p.loc[(df_p2p.loc[:, 'number_females'] > 0) & (df_p2p.loc[:, 'number_males'] > 0), 'borrower_gender_cat'] = 'mixed'
df_p2p.loc[(df_p2p.loc[:, 'number_females'] > 0) & (df_p2p.loc[:, 'number_males'] == 0), 'borrower_gender_cat'] = 'female'
df_p2p.loc[df_p2p.loc[:, 'number_females'] == 0, 'borrower_gender_cat'] = 'male'

df_p2p


,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,number_females,number_males,borrower_count,borrower_gender_cat
0,300.0,300.0,Fruits & Vegetables,Food,Pakistan,PKR,12,12,female,irregular,1,0,1,female
1,575.0,575.0,Rickshaw,Transportation,Pakistan,PKR,11,14,"female, female",irregular,2,0,2,female
2,150.0,150.0,Transportation,Transportation,India,INR,43,6,female,bullet,1,0,1,female
3,200.0,200.0,Embroidery,Arts,Pakistan,PKR,11,8,female,irregular,1,0,1,female
4,400.0,400.0,Milk Sales,Food,Pakistan,PKR,14,16,female,monthly,1,0,1,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,female,monthly,1,0,1,female
671200,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,female,monthly,1,0,1,female
671201,25.0,25.0,Livestock,Agriculture,Kenya,KES,13,1,female,monthly,1,0,1,female
671203,0.0,25.0,Livestock,Agriculture,Kenya,KES,13,0,female,monthly,1,0,1,female


Nach Abschluss des Hinzufügens der cat-Spalte können die Spalten borrower_genders, number_females und number_males gelöscht werden.

Zusätzlich überprüfe ich noch den Datentyp in der neuen Spalte borrower_gender_cat und ändere diese ggf. auf category.

In [160]:
df_p2p.drop(columns=['borrower_genders', 'number_females', 'number_males'], inplace=True)
df_p2p

C:\Users\19Doo\AppData\Local\Temp\ipykernel_9580\4220343798.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,funded_amount,loan_amount,activity,sector,country,currency,term_in_months,lender_count,repayment_interval,borrower_count,borrower_gender_cat
0,300.0,300.0,Fruits & Vegetables,Food,Pakistan,PKR,12,12,irregular,1,female
1,575.0,575.0,Rickshaw,Transportation,Pakistan,PKR,11,14,irregular,2,female
2,150.0,150.0,Transportation,Transportation,India,INR,43,6,bullet,1,female
3,200.0,200.0,Embroidery,Arts,Pakistan,PKR,11,8,irregular,1,female
4,400.0,400.0,Milk Sales,Food,Pakistan,PKR,14,16,monthly,1,female
...,...,...,...,...,...,...,...,...,...,...,...
671199,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,monthly,1,female
671200,0.0,25.0,Livestock,Agriculture,Paraguay,USD,13,0,monthly,1,female
671201,25.0,25.0,Livestock,Agriculture,Kenya,KES,13,1,monthly,1,female
671203,0.0,25.0,Livestock,Agriculture,Kenya,KES,13,0,monthly,1,female


## Kennzahlen

**Anmerkung:**

Je nach Datensatz lassen sich Kennzahlen berechnen: Grundsätzlich können das Verhältnisse, Summen, Differenzen oder komplexere Berechnungen sein.

Unten stehend ein paar Beispiele, wie Kennzahlen als neue Spalte berechnet werden können.

Hier gibts 🔥Lagerfeuer kommt alle her :) Es gibt auch 🧃🍖🍗

In [ ]:
# Hinzufügen ner neuen Spalte 

df.loc[:, 'new_column_name'] = new_values

# Bsp. Quotient, Produkt, Prozentsatz

## Bivariate Datenanalyse

Die Spalten werden auf Korrelation untersucht. Dazu können verschiedene Verfahren verwendet werden. Korrelation bedeutet, dass die Änderungen zweier Variablen miteinander zusammenhängen (positiv oder negativ). _Korrelation ist keine Kausalität!_

**Pearson-Korrelationskoeffizient**

- numerische Werte (int oder float)
- linearer Zusammenhang
- intervallskaliert
- liefert Werte zwischen -1 und 1
- je größer der Wert, desto stärker ist der Zusammenhang

**Beim Verdacht auf nicht-lineare Zusammenhänge bietet sich an, die betroffenen Variablen  zunächst in einem Scatterplot abzutragen. Damit kann man einen ersten Eindruck gewinnen.**

### Kreuztabelle der Korrelationskoeffizienten

In [ ]:
df_corr = df_name.corr(method="pearson", numeric_only=True)
df_corr

Die Kreuztabelle zeigt die Korrelationen zwischen allen numerischen Variablen. Um die Ausprägung besser optisch erfassen zu können, bietet sich eine Heatmap an.

### Heatmap

In [ ]:
heatmap = px.imshow(df_corr,
                    color_continuous_midpoint=0,
                    color_continuous_scale=px.colors.diverging.RdBu_r, # r steht für reversed also der Farbverlauf geht von blau nach rot anstatt von rot nach blau
                    title="Korrelationen",
                    labels={"color":"Korrelationskoeffizient"},
                    height=1000,
                   )
heatmap.update_layout(title_x=0.47)        # Überschrift zentrieren

# Zahlen als Text auf den Zellen platzieren
heatmap.update_layout(annotations=[dict(x=i, y=j, text=str(round(df_corr.iloc[i, j], 2)),
                      showarrow=False, 
                      font=dict(size=10))
                               for i in range(len(df_corr))
                               for j in range(len(df_corr)) 
                                  ])



heatmap.show() 

**Die Variablen mit starker Korrelation zueinander bietet einen ersten Anhalt für weitere Untersuchungen**

## Speichern des DataFrames

**Nach Bereinigung des DataFrames bietet es sich an, diesen vor der EDA separat abzuspeichern**

Dadurch werden die einzelnen Arbeitsabschnitte sauberer voneinander getrennt.
Mit dee EDA in einem separaten Notebook ist es außerdem nicht nur übersichtlicher, sondern auch performanter.

**Um den DataFrame abzuspeichern, bieten sich zwei Formate an:**

- .csv
- pickle

.csv ist universell einsetzbar. Dafür werden Sondereinstellungen (z.B. Formate wie Category oder Verkürzungen wie Int8 statt 64) zurückgesetzt.
Pickle behält diese Einstellungen bei, ist dafür allerdings nur von Python zu öffnen. Außerdem ist nicht garantiert, dass pickle-Dateien über mehrere Python-Versionen hinweg funktionieren. Daher ist bei pickle Vorsicht geboten.

Verwendung einer .csv:

In [ ]:
# df_name.to_csv('df_name_after_preprocessing.csv',sep=',')  # Hier noch auskommentiert, um nicht aus Versehen zu speichern

Verwendung einer pickle:

In [ ]:
# df_name.to_pickle('df_name_after_preprocessing.pkl')   